<a href="https://colab.research.google.com/github/BlackZhc/ML_proj/blob/main/553Proj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Fin 553 Proj2

In [ ]:
import numpy as np
class MDP():
  def __init__(self):
    self.γ = 0.9960
    self.A = [0, 1]
    self.S = [0, 1, 2, 3, 4]

    P0 = np.array([[0.5, .15, .15, 0, .20],
                   [0, .5, .0, .25, .25],
                   [0, 0, .15, .05, .8],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R0 = np.array([0, 0, 0, 10, 0])

    P1 = np.array([[0.5, .25, .15, 0, .10],
                   [0, .5, .0, .35, .15],
                   [0, 0, .20, .05, .75],
                   [0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 1]])

    R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

    self.P = [P0, P1]
    self.R = [R0, R1]

  def step(self, s, a):
    s_prime = np.random.choice(len(self.S), p=self.P[a][s])
    R = self.R[a][s]
    if s_prime == 4:
      done = True
    else:
      done = False
    return s_prime, R, done

  def simulate(self, s, a, π):
    done = False
    t = 0
    history = []
    while not done:
      if t > 0:
        a = π[s]
      s_prime, R, done = self.step(s, a)
      history.append((s, a, R))
      s = s_prime
      t += 1

    return history

In [ ]:
mdp = MDP()

In [ ]:
# Now, starting at phase 0 (state s=0), we will choose to make additional investments (a=1).
s = 0
a = 1

# We feed the state-action pair to the simulator and it tells us what happened in this simulation:
s_prime, R, done = mdp.step(s, a)

print(s_prime, R, done)

1 -0.1 False


In [ ]:
S = np.zeros(5)
q = np.zeros((5,2))
ϵ = 0.01
α = 0.001

def ε_greedy(q, s, ε=0.01):
  a = np.argmax(q[s])

  u = np.random.uniform()
  if u < ε:
    a = np.random.randint(2)
  return a

for iteration in range(1000000):
      # Choose action for today:
  a = ε_greedy(q, s, ε=ε)
  s_prime, R, done = mdp.step(s, a)

    # SARSA update
  a_prime = ε_greedy(q, s_prime, ε=ε)
    # TD_target = R + mdp.γ * q[s_prime, a_prime]

    # Q-learning upate
  TD_target = R + mdp.γ * np.max(q[s_prime, :])

  δ = TD_target - q[s, a]

    # TD update
  q[s, a] = q[s, a] + α * δ

  if done:
    s = 1
  else:
    s = s_prime

    
print(q)

[[ 0.          0.        ]
 [ 5.78455433  6.7131695 ]
 [ 0.          0.        ]
 [10.          6.78438794]
 [ 0.          0.        ]]


In [ ]:
## Part2

In [ ]:
mdp = MDP()
P = mdp.P
R = mdp.R
γ = mdp.γ
S = mdp.S
A = mdp.A
π = [0, 0, 0, 0, 0]


s = 2 # current state
s_prime = 4  # next state
a = 1  # chosen action

# Probability of transition from state s (2) to s_prime (4) if action == a (1):
print(P[a][s, s_prime])

# Reward at state s if action = a
print(R[a][s])

0.75
-0.1


In [ ]:
def construct_Rπ(R, π, S):
  Rπ = np.zeros(len(S))
  for s in S:
    Rπ[s] = R[π[s]][s]
  return Rπ

def construct_Pπ(P, π, S):
  Pπ = np.zeros((len(S), len(S)))
  for s in S:
    for s_prime in S:
      Pπ[s, s_prime] = P[π[s]][s, s_prime]
  return Pπ


def policy_evaluation(π, Vπ):
  Rπ = construct_Rπ(R, π, S)
  Pπ = construct_Pπ(P, π, S)
  for iteration in range(1):
    Vπ = Rπ + γ * Pπ @ Vπ
  return Vπ

def policy_improvement(Vπ):
  Qπ = np.zeros((5, 2))
  π_prime = np.zeros(5, dtype=np.int32)
  for s in S:
    for a in A:
      Qπ[s, a] = R[a][s] + γ * P[a][s] @ Vπ

  # Greedy updates
  for s in S:
    π_prime[s] = np.argmax(Qπ[s, :])
  return π_prime


Vπ = [0, 0, 0, 0, 0]


In [ ]:
for iteration in range(10000):
  Vπ = policy_evaluation(π, Vπ)
  π = policy_improvement(Vπ)

In [ ]:
Vπ

array([ 3.32067538,  6.74501992,  0.58546908, 10.        ,  0.        ])